# Sparkify Data Pipeline

This script runs the Sparkify data pipeline that loads data from S3 into a star schema.

In [ ]:
%load_ext sql

Sets up the Redshift cluster for the data warehouse

In [ ]:
!python3 cluster_setup.py

Executes the drop table and create table statements

In [ ]:
!python3 create_tables.py

Executes the ETL pipeline by loading data from S3 into the staging tables and then from the staging tables into the star schema.

In [ ]:
!python3 etl.py

### Example queries used for validation

In [ ]:
import utils

config = utils.get_config('dwh.cfg')
conn_string="postgresql://{}:{}@{}:{}/{}".format(config.get('CLUSTER', 'db_user'),
                                                 config.get('CLUSTER', 'db_password'),
                                                 config.get('CLUSTER', 'endpoint'),
                                                 config.get('CLUSTER', 'db_port'),
                                                 config.get('CLUSTER', 'db_name'))
%sql $conn_string

Get the top 5 most played songs: the song id, song name, duration, artist name, and number of times it was played.

In [ ]:
%%sql
SELECT
  sp.song_id,
  s.title AS song_name,
  s.duration,
  a.name AS artist_name,
  COUNT(sp.*) AS num_plays
FROM
  songplay AS sp
INNER JOIN
  song AS s
ON
  sp.song_id = s.song_id
INNER JOIN
  artist AS a
ON
  sp.artist_id = a.artist_id
GROUP BY
  1, 2, 3, 4
ORDER BY
  5 DESC
LIMIT
  5
;

Get the top 5 users (by number of songs played): the user_id, their first_name, last_name, number of total song plays, and the number of different songs they have played

In [ ]:
%%sql
SELECT
  sp.user_id,
  u.first_name,
  u.last_name,
  COUNT(sp.*) AS num_plays,
  COUNT(DISTINCT sp.song_id) AS num_songs
FROM
  songplay AS sp
INNER JOIN
  users AS u
ON
  sp.user_id = u.user_id
GROUP BY
  1, 2, 3
ORDER BY
  4 DESC
LIMIT
  5
;

Get the top 5 songs played in 2017: the song_id, song name, artist name, and number of times it was played

In [ ]:
%%sql
SELECT
  sp.song_id,
  s.title AS song_name,
  a.name AS artist_name,
  COUNT(sp.songplay_id) AS num_plays
FROM
  songplay AS sp
INNER JOIN
  song AS s
ON
  sp.song_id = s.song_id
INNER JOIN
  artist AS a
ON
  sp.artist_id = a.artist_id
INNER JOIN
  time AS t
ON
  sp.start_time = t.start_time
WHERE
  t.year = 2018
GROUP BY
  1, 2, 3
ORDER BY
  4 DESC
LIMIT
  5
;

### Cleanup

Run the below command to remove the cluster and IAM role

In [ ]:
!python3 cluster_teardown.py